### Example: O’Reilly Books About Data
A potential investor in DataSciencester thinks data is just a fad. To prove him wrong, you
decide to examine how many data books O’Reilly has published over time. After digging
through its website, you find that it has many pages of data books (and videos), reachable
through 30-items-at-a-time directory pages with URLs like:
```
http://shop.oreilly.com/category/browse-subjects/data.do?
sortby=publicationDate&page=1
```

Unless you want to be a jerk (and unless you want your scraper to get banned), whenever
you want to scrape data from a website you should first check to see if it has some sort of
access policy. Looking at:
```
http://oreilly.com/terms/
```

there seems to be nothing prohibiting this project. In order to be good citizens, we should
also check for a robots.txt file that tells webcrawlers how to behave. The important lines in
```
http://shop.oreilly.com/robots.txt are:
Crawl-delay: 30
Request-rate: 1/30
```
The first tells us that we should wait 30 seconds between requests, the second that we
should request only one page every 30 seconds. So basically they’re two different ways of
saying the same thing. (There are other lines that indicate directories not to scrape, but
they don’t include our URL, so we’re OK there.)

`NOTE`

There’s always the possibility that O’Reilly will at some point revamp its website and break all the logic in
this section. I will do what I can to prevent that, of course, but I don’t have a ton of influence over there.
Although, if every one of you were to convince everyone you know to buy a copy of this book…
To figure out how to extract the data, let’s download one of those pages and feed it to
Beautiful Soup:

In [1]:
!pip install pyppeteer nest_asyncio



In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Path to your WebDriver
driver_path = './chromedriver.exe' 
c_service = webdriver.ChromeService(executable_path=driver_path)
def fetch_page(url, target):
    driver = webdriver.Chrome(service=c_service)
    
    driver.get(url)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, target)))
    
    page_source = driver.page_source
    

    driver.quit()
    
    return page_source

url = 'http://shop.oreilly.com/category/browse-subjects/data.do?sortby=publicationDate&page=1'
target = '.css-ataarr'

page_content = fetch_page(url, target)



In [6]:
soup = BeautifulSoup(page_content, 'html5lib')


In [31]:
filtered = soup.find_all('article' , {'class' : 'css-ataarr'})
types = filtered[0].find_all('div' , {'data-testid' : "ContextLabel/ContentLevels"})
print(filtered[0].find('span' , {'data-testid' : 'ContextLabel/ContentLevels'})) 

None


In [39]:
for item in types:
    print('---------------------------------------------')
    print(item.find('strong').text.split(':')[-1].strip())

---------------------------------------------
Book
